# * 파이썬 코딩의 기술 따라하기

- 예제코드 따라하기
- 중요 문구 / 기억 문구 정리

### - Better Way 1 사용 중인 파이썬의 버전을 알자

commad    
python --version or python3 --version

정리   
python 에는 CPython, Jython, IronPython, PyPy와 같은 다양한 런타임이 있다. 

### - Better Way 2 PEP 8 스타일 가이드를 따르자

url : https://www.python.org/dev/peps/pep-0008/   

#### **화이트스페이스**  
* 탭이 아닌 스페이스로 들여쓴다.
* 문법적으로 의미 있는 들여쓰기는 각 수준마다 스페이스 네 개를 사용한다.
* 한 줄의 문자 길이가 79자 이하여야 한다. 

**명명**
* 함수, 변수, 속성은 lowercase_underscore 형식을 따른다.
* 보호 인스턴스 속성은 _leading_underscore 형식을 따른다.

**표현식과 문장**
* 길이를 확인하여 빈 값을 확인하고자 않는다.
if (len(somelist) == 0) 대신 if not somelist 사용

### - Better Way 3 bytes, str, unicode의 차이점을 알자 
파이썬 3에서는 bytes 와 str 두 가지 타입으로 문자 시퀀스를 나타낸다.  
bytes 인스턴스는 raw 8비트 값을 저장한다. str 인스턴스는 유니코드 문자를 저장한다.  

파이썬 3에서는 먼저 str이나 bytes를 입력으로 받고 str을 반환하는 메서드가 필요하다.

In [9]:
# ex) str이나 bytes를 입력으로 받고 str을 반환하는 메서드
# bytes code type example ) b'\x80abc'
def to_str(bytes_or_str):
    if isinstance(bytes_or_str, bytes): # 인스턴스와 클래스를 인자로 받아 인스터스가 해당 클래스의 인스턴스인지 확인
        value = bytes_or_str.decode('utf-8', 'ignore')
    else:
        value = bytes_or_str
    return value # str 인스턴스

In [10]:
print(to_str(b'\x80abc'))

abc


In [11]:
# ex 2) str이나 bytes를 받고 bytes를 반환하는 메서드도 필요하다.
def to_bytes(bytes_or_str):
    if isinstance(bytes_or_str, str):
        value = bytes_or_str.encode('utf-8')
    else:
        value = bytes_or_str
    return value

print(to_bytes('abc'))

b'abc'


### - Better Way 4 복잡한 표현식 대신 헬퍼 함수를 작성하자
  
파이썬의 간결한 문법을 이용하면 많은 로직을 표현식 한 줄로 쉽게 작성할 수 있다.  
예를 들어 URL에서 쿼리 문자열을 디코드해야 한다고 하자. 다음예에서 각 쿼리 문자열 파라미터는 정수 값을 표현한다.

In [12]:
from urllib.parse import parse_qs
my_values = parse_qs('red=5&blue=0&green=', keep_blank_values=True)
print(repr(my_values))

{'red': ['5'], 'blue': ['0'], 'green': ['']}


In [16]:
# 존재하지만 값이 비어 있는 경우 => 디폴트 값 0 지정 
print('Red:    ', my_values.get('red'))
print('green:    ', my_values.get('green'))
print('opacity:    ', my_values.get('opacity'))

print('---------------------')
red = my_values.get('red', [''])
red = int(red[0]) if red[0] else 0

# more better => make helper function
def get_first_init(values, key, default=0):
    found = values.get(key, [''])
    if found[0]:
        found = int(found[0])
    else:
        found = default
    return found

green = print('green:    ', get_first_init(my_values, 'green'))

Red:     ['5']
green:     ['']
opacity:     None
---------------------
green:     0


### - Better Way 5 시퀀스를 슬라이스하는 방법을 알자 (중요)

파이썬은 시퀀스를 슬라이스해서 조각으로 만드는 문법을 제공한다.  
이렇게 슬라이스하면 최소한의 노력으로 시퀀스 아이템의 부분집합(subset)에 접근할 수 있다.  
대상은 내장 타입이 list, str, bytes이다. __getitem__, __setitem__ 이라는 특별한 메서드를 구현하는 파이썬의 클래스에도 슬라이싱을 적용할 수 있다.  

**가장 기본적인 포멧:** somelist[start:end] 여기서 start 인덱스는 포함, end 인덱스는 제외이다.

In [19]:
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print('First four: ', a[:4])
print('last four: ', a[-4:])
print('Middle Two: ', a[3:-3])

First four:  ['a', 'b', 'c', 'd']
last four:  ['e', 'f', 'g', 'h']
Middle Two:  ['d', 'e']


In [21]:
assert a[:5] == a[0:5]
assert a[5:] == a[5:len(a)]

In [23]:
a[:]
a[:5]

['a', 'b', 'c', 'd', 'e']

#### Note 
리스트의 인덱스를 음수 변수로 지정하면 슬라이싱으로 뜻밖의 결과를 얻는 몇 가지 상황이 있다. 그 중 n이 0일때 somelist[-0:]이 되면 원본 리스트의 복사본을 만든다.

In [24]:
print('Before: ', a)
a[2:7] = [99, 22, 14]
print('After: ', a)

Before:  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
After:  ['a', 'b', 99, 22, 14, 'h']


### - Better Way 6 한 슬라이스에 start, end , stride를 함께 쓰지 말자

파이썬에는 somelist[start:end:stride]처럼 슬라이스의 스트라이드(간격)을 설정하는 특별한 문법도 있다. 이 문법을 이용하면 시퀀스를 슬라이스할 대 매 n 번째 아이템을 가져올 수 있다.

In [25]:
a = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
odds = a[::2]
evens = a[1::2]
print(odds)
print(evens)

['red', 'yellow', 'blue']
['orange', 'green', 'purple']


In [27]:
# stride를 쓰려면 start나 end 중 하나와 같이 쓰자
a[1::-1]

['orange', 'red']

### - Better Way 7 map과 filter대신 컴프리헨션을 사용하자.

파이썬에는 한 리스트에서 다른 리스트를 만들어내는 간결한 문법이 있다.  
이 문법을 사용한 표현식을 리스트 컴피르헨션이라고 한다.

In [32]:
# ex 1) 리스트의 각 원소를 제곱
a = [i for i in range(1,11)]
# 리스트 컴프리헨션
squares = [x ** 2 for x in a]
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [35]:
# map 이용 => 람다 함수를 생성해야 해서 깔끔해보이지 않는다.
squares2 = map(lambda x: x ** 2, a)
print(squares2)

In [42]:
# even squares
even_squares = [x ** 2 for x in a if x % 2==0]
print(even_squares)

# map 사용
alt = map(lambda x: x**2, filter(lambda x: x % 2 == 0, a))
assert even_squares == list(alt)

[4, 16, 36, 64, 100]


***딕셔너리와 세트에도 리스트 컴프리헨션에 해당하는 문법이 있다.  
컴프리헨션 문법을 쓰면 알고리즘을 작성할 때 파생되는 자료 구조를 쉽게 생성할 수 있다**

In [43]:
# ex )
chile_ranks = {'ghost': 1, 'habanero':2, 'cayeene': 3}
rank_dict = {rank: name for name, rank in chile_ranks.items()}
chile_len_set = {len(name) for name in rank_dict.values()}
print(rank_dict)
print(chile_len_set)

{1: 'ghost', 2: 'habanero', 3: 'cayeene'}
{8, 5, 7}


### - Better Way 8 리스트 컴프리헨션에서 표현식을 두 개 넘게 쓰지 말자 

리스트 컴프리헨션은 다중 루프도 지원한다. 

In [44]:
# ex) 
matrix = [[1,2,3], [4,5,6], [7,8,9]]
flat = [x for row in matrix for x in row]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [46]:
# 2차원 행렬 구성하기
squared = [[x**2 for x in row] for row in matrix]
print(squared)

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]


In [48]:
# ex ) 다중 if 조건
a = [i for i in range(1,11)]
b = [x for x in a if x > 4 if x % 2 == 0]
c = [x for x in a if x > 4 if x % 2 == 0]
b

[6, 8, 10]

**조건은 루프의 각 레벨에서 for 표현식 뒤에 설정할 수 있다.  
예를 들어 행렬에서 로우의 합이 10 이상이고 3으로 나누어 떨어지는 셀을 구한다고 하자. 다음처럼 리스트 컴프리헨션으로 표현하면 간단하지만 이해하기가 매우 어렵다.**

### **결론: 표현식이 두 개를 넘어가면 컴프리헨션은 피하자!**

### - Better Way 9 컴프리헨션이 클 때는 제너레이터 표현식을 고려하자.

리스트 컴프리헨션의 문제점은 입력 시퀀스에 있는 각 값별로 아이템을 하나씩 담은 새 리스트를 통째로 생성한다는 점이다.  
입력이 적을 때는 괜찮지만 클 때는 메모리를 많이 소모해서 프로그램을 망가뜨리는 원인이 되기도 한다.  


예를 들어 파일을 읽고 각 줄에 있는 문자의 개수를 반환한다고 하자.  
이 작업을 리스트 컴프리헨션으로 하면 파일에 있는 각 줄의 길이만큼 메모리가 필요하다.  
파일에 오류가 있거나 끊김이 없는 네트워크 소켓일 경우 리스트 컴프리헨션을 사용하면 문제가 발생한다.  
다음은 입력값이 적은 경우만 처리할 수 있는 방식으로 리스트 컴프리헨션을 사용한 예다.  

In [49]:
value = [len(x) for x in open('/tmp/my_file.txt')]
print(value)

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/my_file.txt'

**파이썬은 이 문제를 해결하려고 리스트 컴프리핸션과 제너레이터를 일반화한 제너레이터 표현식을 제공한다.  
제너레이터 표현식은 실행될 때 출력 시퀀스를 모두 구체화(여기서는 메모리에 로딩)하지 않는다.  
대신에 표현식에서 한 번에 한 아이템을 내주는 이테레이터로 평가된다.**

In [50]:
# generator 표현식은 () 문자 사이에 리스트 컴프리헨션과 비슷한 문법을 사용하여 생성
it (len(x) for x in open('/tmp/my_file.txt'))
print(it)
print(next(it))
print(next(it))

NameError: name 'it' is not defined

**제너레이터 표현식의 또 다른 강력한 결과는 다른 제너레이터 표현식과 함께 사용할 수 있다는 점이다.**

In [51]:
roots = ((x, x**0.5) for x in it)
print(next(roots))

NameError: name 'it' is not defined

### - Better Way 10 range 보다는 enumerate를 사용하자.

내장함수 range는 정수 집합을 순회하는 루플를 실행할때 유용하다

In [53]:
random_bits = 0
for i in range(64):
    if randint(0, 1):
        random_bits |=1 << i

NameError: name 'randint' is not defined

In [55]:
# 리스트 길이 만큼 순회가기 
# range 사용
flavor_list = ['vanila', 'chocolate', 'pecan', 'strawberry']
for i in range(len(flavor_list)):
    flavor = flavor_list[i]
    print('%d: %s' % (i+1, flavor))

# 위의 코드는 리스트의 길이를 알아내야 하고, 배열을 인덱스로 접근해야 하며, 읽기 불편하다.
print('----------------')
# enumerate 사용
for i, flavor in enumerate(flavor_list):
    print('%d: %s' % (i+1, flavor))

1: vanila
2: chocolate
3: pecan
4: strawberry
----------------
1: vanila
2: chocolate
3: pecan
4: strawberry


In [57]:
# enumerate로 세기 시작할 숫자를 지정하면 코드를 더 짧게 만들 수 있다.
for i, flavor in enumerate(flavor_list, 1):
    print('%d: %s' % (i, flavor))

1: vanila
2: chocolate
3: pecan
4: strawberry


### - Better Way 11 이터레이터를 병렬로 처리하려면 zip을 사용하자.

파이썬에서 관련 객체로 구성된 리스트를 많이 사용한다는 사실은 쉽게 알 수 있다.  
리스트 컴프리헨션을 사용하면 소스 리스트에 표현식을 적용하여 파생 리스트를 쉽게 얻을 수 있다.

In [59]:
names = ['Cecilia', 'Lise', 'Marie']
letters = [len(n) for n in names]

'''
    파생 리스트의 아이템과 소스 리스트의 아이템은 서로의 인덱스로 연관되어 있다.
    따라서 두 리스트를 병렬로 순회하려면 소스 리스트인 names의 길이만큼 순회하면 된다.
'''

longest_name = None
max_letters = 0

for i in range(len(names)):
    count = letters[i]
    if count > max_letters:
        longest_name = names[i]
        max_letters = count

print(longest_name)

Cecilia


In [60]:
'''
    위 예제의 문제는 전체 루프문이 별로 보기 안좋다는 것이다. 
    names와 lettesr를 인덱스로 접근하면 코드를 읽기 어려워진다. 
    루프의 인덱스 i로 배열에 접근하는 동작이 두 번 일어난다.
    enumerate를 사용하면 이런 문제점을 약간 개선할 수 있지만, 여전히 완벽하지는 않다.
'''
for i, name in enumerate(names):
    count = letters[i]
    if count >  max_letters:
        longet_name = name
        max_letters = count

'''
    파이썬은 위의 코드를 좀 더 명료하게 하는 내장함수 zip을 제공한다. 
    파이썬 3에서 zip은 지연 제너레이터로 이터레이터 두 개 이상을 감싼다.
    zip 제너레이터는 각 이터레이터로부터 다음 값을 담은 튜플을 얻어온다.
    zip 제너레이터를 사용한 코드는 다중 리스트에서 인덱스로 접근하는 코드보다 훨씬 명료하다. 
'''

for name, count in zip(names, letters):
    if count > max_letters:
        lognest_name = name
        max_letters = count

In [62]:
list(zip(names, letters))

[('Cecilia', 7), ('Lise', 4), ('Marie', 5)]

**zip으로 실행할 리스트의 길이가 같다고 확신 할 수 없다면 내장 모듈 itertools의 zip_longest를 사용하는 방안을 고려해보자**

### - Better Way 12 for와 while 루프 뒤에는 else 블록을 쓰지 말자

파이썬의 루프에는 대부분의 다른 프로그래밍 언어에서 없는 추가적인 기능이 있다.
루프에서 반복되는 내부 블록 다음에 else 블록을 둘 수 있는 기능이다.

In [63]:
for i in range(3):
    print('Loop %d' % i)
else: 
    print('Else block!')

Loop 0
Loop 1
Loop 2
Else block!


elae 블록은 루프가 종료되자마자 실행된다.  
루프에서 break문을 사용해야 else 블록을 건너 뛸 수 있다.

In [64]:
for i in range(3):
    print('Loop: %d' % i)
    if i == 1:
        break
else:
    print('Else Block')

Loop: 0
Loop: 1


다른 놀랄 만한 점은 빈 시퀀스를 처리하는 루프문에서도 else 블록이 즉시 실행된다는 것이다.

In [65]:
for x in []:
    print("Never runs")
else:
    print('For Else Block!')

For Else Block!


In [66]:
# else 블록은 while 루프가 처음부터 거짓인 경우에도 실행된다.

while False:
    print('Never runs')
else:
    print('While else block')

While else block


**루프 뒤에 else 블록을 사용하면 직관적이지 않고 혼동하기 쉬우니 사용하지 말아야 한다.**

### - Better Way 13 try / except / else / finality에서 각 블록의 장점을 이용하자

파이썬에는 예외 처리 과정에서 동작을 넣을 수 있는 네 번의 구분되는 시점이 있다.

In [67]:
# finally 블록
# 예외를 전달하고 싶지만, 예외가 발생해도 정리 코드를 실행하고 싶을 때 try / finally 사용하면 된다.

handle = open('/tmp/random_data.txt') # IOError가 일어날 수 있음

try:
    data = handle.read()
finally:
    handle.close() # try: 이후에 항상 실행됨.

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/random_data.txt'

In [68]:
# else 블록
# 코드에서 어떤 예외를 처리하고 어떤 예외를 전달할지를 명확하게 하려면 try / except / else를 사용해야 한다.
# try 블록이 예외를 일으키지 않으면 else 블록이 실행된다. else 블록을 사용하면 try 블록의 코드를 최소로 줄이고 가독성을 높일 수 있다.

def load_json_key(data, key):
    try:
        result_dict = json.loads(data) # ValueError가 일어날 수 있음
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key] # keyError가 일어날 수 있음

In [69]:
# 모두 함께 사용하기
# 복합문 하나로 모든 것을 처리하고 싶다면 try / except / else / finally를 사용하면 된다.

UNDEFIEND = object()
def divide_json(path):
    handle = open(path, 'r+')
    try:
        data = handle.read() # UnicodeDecodeError가 일어날 수 있음
        op = json.loads(data) # ValueError가 일어날 수 있음
        value = (
            op['numerator'] /
            op['denominator'] # ZerorDivisionError가 일어날 수 있음
        )
    except ZerorDvivisonError as e:
        return UNDEFIEND
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)
        handlre.write(result)
        return value
    finally:
        handle.close()

## 함수

### - Better Way 14 None을 반환하기보다는 예외를 일으키자.

파이썬 프로그래머들은 유틸리티 함수를 작성할 때 반환 값  None에 특별한 의미를 부여하는 경향이 있다.  
어떤 경우에는 일리 있어 보인다. 예를 들어 어떤 숫자를 다른 숫자로 나누는 헬퍼 함수를 생각해보자. 0으로 나누는 경우에는 결과가 정의되어 있지 않기때문에 None을 반환하는게 자연스럽다.

In [71]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

# 이 함수를 사용하는 코드는 반환 값을 다음과 같이 해석한다.

x, y = 0, 5
result = divide(x, y)
if result is None:
    print('Invalid inputs')

In [72]:
def divide(a,b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid Inputs') from e

x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print('invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 2.5


### - Better Way 15 클로저가 변수 스코프와 상호 작용하는 방법을 알자

숫자 리스트를 정렬할 때 특정 그룹의 숫자들이 먼저 오도록 우선순위를 매기려고 한다고 하자.  
이런 패턴은 사용자 인터페이스를 표현하거나, 다른 것보다 중요한 메시지거나 예외 이벤트를 먼저 보여줘야 할 때 유용하다.  

이렇게 만드는 일반적인 방법은 리스트의 sort 메서드에 헬퍼 함수를 key 인수로 넘기는 것이다. 헬퍼의 반환 값은 리스트에 있는 각 아이템을 정렬하는 값으로 사용된다. 헬퍼는 주어진 아이템이 중요한 그룹에 있는지 확인하고 그에 따라 정렬키를 다르게 할 수 있다.

In [73]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    
# 이 함수는 간단한 입력값에 사용한다.

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


**1) 파이썬은 클로저를 지원한다. 클로저란 자신이 정의된 스코프에 있는 변수를 참조하는 함수다. 바로 이 점 덕분에 helper 함수가 sort_priory의 group 인수에 접근할 수 있다.**

**2) 함수는 파이썬에서 일급 객체다. 이 말은 함수를 직접 참조하고, 변수에 할당하고, 다른 함수의 인수로 전달하고, 표현식과 if문 등에서 비교할 수 있다는 의미다. 따라서 sort 메서드에서 클로저 함수를 key 인수로 받을 수 있다.**

**3) 파이썬에는 튜플을 비교하는 특정한 규칙이 있다. 먼저 인덱스 0으로 아이템을 비교하고 그 다음으로 인덱스 1, 다음은 인덱스 2와 같이 진행한다. helper 클로저의 반환 값이 정렬 순서를 분리된 두 그룹으로 나뉘게 한 건 이 규칙때문이다.**


In [75]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

found = sort_priority2(numbers, group)
print('Found: ', found)
print(numbers)

Found:  False
[2, 3, 5, 7, 1, 4, 6, 8]


정렬된 결과는 올바르지만 found 결과는 틀렸다.  
group에 속한 아이템을 numbers에서 찾을 수 있었지만 함수는 False를 반환했다.  
어째서 이런 일이 일어났을까 ??  

표현식에서 변수를 참조할 때 파이썬 인터프리터는 참조를 해결할려고 다음과 같은 순서로 스코프를 탐색한다.  

**1)** 현재 함수의 스코프  
**2)** (현재 스코프를 담고 있는 다른 함수 같은) 감싸고 있는 스코프  
**3)** 코드를 포함하고 있는 모듈의 스코프(전역 스코프라고도 함)
**4)** (len이나 str 같은 함수를 담고 있는) 내장 스코프  

이 중 어느 스코프에도 참조한 이름으로 된 변수가 정의되어 있지 않으면 NameError가 일어난다.  

위와 같은 일이 발생한 것은
인터프리터가 첫 번째 found=False 변수를 sort_priority2에서 정의된 변수로  
두번째 found=True를 helper 함수 내에서 새롭게 정의된 변수로 본 것이다. 

In [76]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True